# Использование метода ближайших соседей для решения задачи классификации
Группа: ИВТ-М20.<br/>
Студент: Лискунов Роман Геннадьвич

In [28]:
import warnings
from typing import Union, Any

warnings.filterwarnings('ignore')

In [29]:
from pandas import (
    DataFrame,
    read_csv
)
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

## Чтение данных из файлов (1)

In [30]:
target: DataFrame = DataFrame(read_csv('data/target5.csv', header=0))
target.columns = ['data']
var: DataFrame = DataFrame(read_csv('data/var5.csv', sep=',', header=None))


## Транспонирование исходной матрицы (2)
## Отображение нескольких записей (3)

In [31]:
targetT: DataFrame = target.T
varT: DataFrame = var.T
print(targetT.head())
print(targetT.tail())
print(varT.head())
print(varT.tail())

      0   1   2   3   4   5   6   7   8   9   ...  20  21  22  23  24  25  26  \
data   1   1   1   1   1   1   1   1   1   1  ...   2   2   2   2   2   2   2   

      27  28  29  
data   2   2   2  

[1 rows x 30 columns]
      0   1   2   3   4   5   6   7   8   9   ...  20  21  22  23  24  25  26  \
data   1   1   1   1   1   1   1   1   1   1  ...   2   2   2   2   2   2   2   

      27  28  29  
data   2   2   2  

[1 rows x 30 columns]
          0          1          2          3          4          5   \
0  10.659607   5.623670   5.773385   8.318744  10.798757   3.392248   
1  12.707813  12.437502  12.293281  15.946799  15.800968  18.957569   
2   4.944295   7.421738   7.322419  11.696521   8.865894  15.108460   
3  12.466091   6.826808   8.880880   7.573538  13.083592   9.473492   
4   6.401173   6.063046   6.493885   6.441410   7.014855   7.835656   

          6          7          8          9   ...         20         21  \
0   9.309021   9.674849   7.147078   5.687350  ..

## Обучающаяся и проверочная выборка (4)

In [32]:
print(target.shape, var.shape)
target_train, target_test, var_train, var_test = train_test_split(target, var, test_size=0.3, train_size=0.7)
print(target_test.shape, var_test.shape)
print(target_train.shape, var_train.shape)

(30, 1) (30, 200)
(9, 1) (9, 200)
(21, 1) (21, 200)


## Алгоритм K-ближайших соседей (5)

In [33]:
knn: KNeighborsClassifier = KNeighborsClassifier(n_neighbors=3)
knn.fit(var_train, target_train)
scores: object = cross_val_score(knn, var_train, target_train, cv=10, scoring='accuracy')
print(f"Test score {scores}")
y_true: object = target_test
y_pred: object = knn.predict(var_test)
print(y_pred);

Test score [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1 1 2 1 2 2 1 1 2]


## Оценка качества (6)

In [34]:
print(accuracy_score(y_true, y_pred, normalize=True))
print(classification_report(y_true, y_pred))

0.8888888888888888
              precision    recall  f1-score   support

           1       1.00      0.83      0.91         6
           2       0.75      1.00      0.86         3

    accuracy                           0.89         9
   macro avg       0.88      0.92      0.88         9
weighted avg       0.92      0.89      0.89         9



## Отбор информативных частот (7)

## Подбор гиперпараметров (8)


In [35]:
parameters: dict[str, Union[list[int], list[str]]] = {
    'n_neighbors': [3, 5, 11, 19],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}
grid: GridSearchCV = GridSearchCV(
    KNeighborsClassifier(),
    parameters,
    cv=5
)
gs_result: Union[GridSearchCV, Any] = grid.fit(var_train, target_train)
print(gs_result)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': [3, 5, 11, 19],
                         'weights': ['uniform', 'distance']})


## Повторное обучение (9)

In [36]:
print(gs_result.best_params_)
print(gs_result.best_score_)
print(gs_result.predict(var_test))

y_pred = gs_result.predict(var_test)
print(round(accuracy_score(y_true, y_pred, normalize=True), 3))

{'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'uniform'}
1.0
[1 1 2 1 2 2 1 1 2]
0.8888888888888888


# Ответы на контрольные вопросы

## 1. Опишите этапы реализации алгоритма KNN и для решения каких задач его можно использовать?
### Этапы реализации
Алгоритм K-ближайших соседей (KNN) использует «сходство признаков» для прогнозирования значений новых точек данных, что также означает, что новой точке данных будет присвоено значение на основе того, насколько близко он соответствует точкам в обучающем наборе. Мы можем понять его работу с помощью следующих шагов —

Шаг 1 — Для реализации любого алгоритма нам нужен набор данных. Таким образом, во время первого шага KNN мы должны загрузить данные обучения, а также данные испытаний.

Шаг 2 — Далее нам нужно выбрать значение K, то есть ближайшие точки данных. K может быть любым целым числом.

Шаг 3 — Для каждой точки в тестовых данных сделайте следующее —

3.1 — Рассчитайте расстояние между данными испытаний и каждой строкой данных тренировки с помощью любого из методов, а именно: Евклидово, Манхэттенское или Хэмминговское расстояние. Наиболее часто используемый метод расчета расстояния — евклидов.

3.2 — Теперь, основываясь на значении расстояния, отсортируйте их в порядке возрастания.

3.3 — Далее он выберет верхние K строк из отсортированного массива.

3.4 — Теперь он назначит класс контрольной точке на основе наиболее часто встречающегося класса этих строк.

3.1 — Рассчитайте расстояние между данными испытаний и каждой строкой данных тренировки с помощью любого из методов, а именно: Евклидово, Манхэттенское или Хэмминговское расстояние. Наиболее часто используемый метод расчета расстояния — евклидов.

3.2 — Теперь, основываясь на значении расстояния, отсортируйте их в порядке возрастания.

3.3 — Далее он выберет верхние K строк из отсортированного массива.

3.4 — Теперь он назначит класс контрольной точке на основе наиболее часто встречающегося класса этих строк.

Шаг 4 — Конец
### Области применения алгоритма KNN
Алгоритм k-ближайших соседей имеет широкое применение. Например:

- Обнаружение мошенничества. Новые случаи мошенничества могут быть похожи на те, которые происходили когда-то в прошлом. Алгоритм KNN может распознать их для дальнейшего рассмотрения.
- Предсказание отклика клиентов. Можно определить отклик новых клиентов по данным из прошлого.
- Медицина. Алгоритм может классифицировать пациентов по разным показателям, основываясь на данных прошедших периодов.
- Прочие задачи, требующие классификацию.

## 2. Какие метрики используются для оценки качества работы алгоритмов классификации? Опишите данные метрики с математической точки зрения и скажите, чем они отличаются друг от друга.
[Подробно](https://goo-gl.ru/MlQae "Оценка качества")

## 3. Каким способом можно повысить качество работы алгоритмов классификации?
